In [1]:
from scipy.io import loadmat
import numpy as np
from scipy.stats import mode
from sklearn.preprocessing import PowerTransformer
import pickle
from tqdm import tqdm
from multiprocessing import Pool
import os

2023-11-19 15:41:00.909082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 15:41:00.909160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 15:41:00.909201: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def pickle_pi_set(pi):
    pi_set = []
    mat_i = loadmat(f"data/mats/data-science-P{pi+1}.mat")
    coord_to_col = mat_i["meta"][0][0][8]
    data = mat_i["data"]
    info = mat_i["info"][0]

    yeojohnTr = PowerTransformer(standardize=True)
    
    for i in tqdm(range(360), desc=str(pi)):
        datum = data[i][0][0]

        scan = np.zeros((23, 61, 51))
        for x in range(51):
            for y in range(61):
                for z in range(23):
                    scan[z, y, x] = datum[coord_to_col[x, y, z] - 1]


        mask = scan != mode(scan.flat).mode

        scan[mask] = yeojohnTr.fit_transform(scan[mask].reshape((-1, 1))).reshape((-1, ))
        scan[scan == mode(scan.flat).mode] = scan.min()
        scan[scan < -3] = -3
        scan[scan > 3] = 3
        scan /= 3

        pi_set.append((scan, info[i][2][0], info[i][0][0]))

    with open(f"data/pickles/{pi}.pkl", "wb") as f:
        pickle.dump(pi_set, f)
        

In [3]:
def pickle_set():
    if not os.path.exists("data/pickles"):
        os.mkdir("data/pickles")
    Pool(processes=5).map(pickle_pi_set, range(9))

pickle_set()

5: 100%|██████████| 360/360 [01:12<00:00,  4.95it/s]
